In [279]:
# !pip install pandas
# !pip install matplotlib
# !pip install scipy
# !pip install codac
# !pip install trustregion
# !pip install networkx


In [280]:
from codac import Interval, IntervalVector, CtcDist, Function,CtcFunction
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import deque
import networkx as nx
from scipy.optimize import minimize, minimize_scalar
from scipy.spatial.transform import Rotation as Rot
import math as math
import multiprocessing
from itertools import product, chain

N_ATOMS = 63

In [281]:
f= open("graph_02.txt", mode="r")
li = []
for line in f:
    li.append(line)

In [282]:
f.close()

In [283]:
# def get_f_ctc(sub1, sub2, dico_interval, point, axis):
#     li_ctc = []
#     li_constrains = []
#     li_f = []

#     #pour verifier
#     axis /= np.linalg.norm(axis)

#     for atom1 in sub1.sub.nodes:
#         for atom2 in sub2.sub.nodes:
#             if atom1 == atom2:
#                 continue
#             try:
#                 mini, maxi = dico_interval[(i,j)][0], dico_interval[(i,j)][1]
#             except KeyError:
#                 continue
            
#             #define_domaine
#             low = 0
#             up = np.pi * 2
#             li_constrains.append([low,up])

#             vector1 = sub1.X[sub1.index_to_X[atom1]]
#             vector2 = sub2.X[sub2.index_to_X[atom2]]

#             vector2_unchanged = point + (np.dot(vector2, axis)*axis)
#             vector2_changed = vector2 - vector2_unchanged
#             #la ta fonction elle calcule la taille entre l'atome 1 et l atome 2 si la rotation d'angle theta est effectué,
#             #jsp comment tu peux/veux ajouter la contrainte sur mini et maxi mais en tout cas tout est la 
#             f = Function("theta", f"sqrt(sqr(
#                             {vector2_unchanged[0]} + 
#                             cos(theta){axis[2]}*cos(theta * {axis[1]})*{vector2_changed[0]} + 
#                             (cos(theta * {axis[2]})*sin(theta * {axis[1]})*sin(theta * {axis[0]}) - sin(theta * {axis[2]})*cos(theta * {axis[0]}))*{vector2_changed[1]} +
#                             (cos(theta * {axis[2]})*sin(theta * {axis[1]})*sin(theta * {axis[0]}) + sin(theta * {axis[2]})*sin(theta * {axis[0]}))*{vector2_changed[2]} +
#                             -{vector1[0]})+sqr(
#                             {vector2_unchanged[1]} + 
#                             sin(theta * {axis[2]})*cos(theta * {axis[1]})*{vector2_changed[0]} + 
#                             (sin(theta * {axis[2]})*sin(theta * {axis[1]})*sin(theta * {axis[0]}) + cos(theta * {axis[2]})*cos(theta * {axis[0]}))*{vector2_changed[1]} +
#                             (sin(theta * {axis[2]})*sin(theta * {axis[1]})*sin(theta * {axis[0]}) - cos(theta * {axis[2]})*sin(theta * {axis[0]}))*{vector2_changed[2]} +
#                             -{vector1[1]})+sqr(
#                             {vector2_unchanged[2]} + 
#                             -sin(theta * {axis[1]})*{vector2_changed[0]} + 
#                             cos(theta * {axis[1]})*sin(theta * {axis[0]})*{vector2_changed[1]} +
#                             cos(theta * {axis[1]})*cos(theta * {axis[0]})*{vector2_changed[2]} +                            
#                             -{vector1[2]}))")
#             li_f.append(f)
#         contractor = CtcFunction(f, Interval(low,up))
#         li_ctc.append(contractor)

# return li_ctc,li_f, li_constrains


## Marco Part, let's rock it baby

In [284]:
dico_constrains = {}
dico_interval = {}
dico_init_inter = {}
for elem in li:
    i = int(elem[6:8].split(" ")[-1])-1
    j = int(elem[16:18].split(" ")[-1])-1
    mini = float(elem[21:42])
    maxi = float(elem[45:-1])
    dico_constrains[(i,j)] = (maxi + mini)/2
    dico_constrains[(j,i)] = (maxi + mini)/2
    dico_interval[(i,j)] = [mini, maxi]
    dico_interval[(j,i)] = [mini, maxi]
    dico_init_inter[(i,j)] = [mini, maxi]
    dico_init_inter[(j,i)] = [mini, maxi]


In [285]:
if True:
    def get_edm(nodes,t, graph):
        n = len(nodes)
        matrix = np.zeros((n,n))
        for k in range(n-1):
            for l in range(k+1,n):
                i = nodes[k]
                j = nodes[l]
                low = graph.get_edge_data(i,j)["mini"]
                up = graph.get_edge_data(i,j)["maxi"]
                matrix[k,l] = (1-t[k,l])*low + t[k,l]*up
                matrix[l,k] = (1-t[k,l])*low + t[k,l]*up
        return matrix

    def get_x(EDM):
        U, S, Vh = np.linalg.svd(EDM)
        x = U[:,:3]
        return x
    lamb = 1
    tau = 0.1

    def theta(x,i,j,c):
        return (lamb**2 *(c-np.sqrt(np.linalg.norm(x[3*i:3*(i+1)]-x[3*j:3*(j+1)])**2 + tau**2))**2 + tau**2)

    def phi(x,i,j, low,up):
        value = lamb*(low - up) + theta(x,i,j,low) + theta(x,i,j,up)
        return value

    def cost(x,nodes,graph):
        n = len(nodes)
        cost_value = 0
        for k in range(n-1):
            for l in range(k+1,n):
                i = nodes[k]
                j = nodes[l]
                low = graph.get_edge_data(i,j)["mini"]
                up = graph.get_edge_data(i,j)["maxi"]
                cost_value += phi(x,k,l,low,up)
        return cost_value

    def get_position_sub(sub):
        t = np.ones((len(sub),len(sub))) * 0.7
        D = get_edm(list(sub.nodes),t,sub)
        x = get_x(D)
        res = minimize(cost,x.flatten(),args=(list(sub.nodes),sub),method='BFGS')
        X = res.x.reshape(len(sub),3)
        return X

    def get_dico_sub(sub):
        dico_index_to_X = {}
        dico_X_to_index = {}
        count = 0
        for node in list(sub.nodes):
            dico_index_to_X[node] = count
            count += 1
        count = 0
        for node in list(sub.nodes):
            dico_X_to_index[count] = node
            count += 1
        return dico_index_to_X, dico_X_to_index

    def visualize_step(list_sub, dico_interval, vision=True):
        list_atomes_interested = []
        for i in list_sub:
            print(i.sub.nodes)
            i.test_constraint(dico_interval, approx=vision)
            list_atomes_interested += list(i.sub.nodes)
        print(f'{len(list_atomes_interested)} atomes for {len(set(list_atomes_interested))} different ones. {len(list_atomes_interested) - len(set(list_atomes_interested))} atome(s) to merge ')


In [286]:
G = nx.Graph()
G.add_nodes_from([i for i in range(N_ATOMS)])
for (i,j), (mini, maxi) in dico_interval.items():
    G.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])


In [287]:
def align_figures(figure1, figure2, point1, point2):
    # Calculer les vecteurs de translation pour chaque point en commun
    vector = figure1[point1] - figure2[point2]

    return vector

In [288]:
def calculate_rotation_matrix(fixed_point, target_point, desired_position, verbose):
    # Calculate vectors from fixed_point to target_point and fixed_point to desired_position
    vector_target = target_point - fixed_point
    vector_desired = desired_position - fixed_point

    if verbose > 1:
        print(f"test_lenght : {np.linalg.norm(vector_target)} , {np.linalg.norm(vector_desired)}")
    
    # Calculate the rotation axis and angle
    rotation_axis = np.cross(vector_target, vector_desired)
    rotation_axis /= np.linalg.norm(rotation_axis)

    #atan2(crossproduct.length,scalarproduct)
    angle = np.arctan2(np.linalg.norm(np.cross(vector_target, vector_desired)), np.dot(vector_target, vector_desired))

    # Create the rotation matrix
    rotation_matrix = Rot.from_rotvec(angle * rotation_axis)
    
    return rotation_matrix


In [289]:
def calculate_sec_rotation_matrix(segment_point1, segment_point2, target_point, desired_position, verbose):
    # Calculate vectors along the line segment
    vector_segment = segment_point2 - segment_point1
    vector_segment /= np.linalg.norm(vector_segment)

    vector_target = target_point - np.dot(target_point,vector_segment) * vector_segment
    vector_desired = desired_position - np.dot(desired_position,vector_segment) * vector_segment
    point_rotation = segment_point1 - np.dot(segment_point1, vector_segment) * vector_segment

    return calculate_rotation_matrix(point_rotation, vector_target, vector_desired, verbose), vector_segment, point_rotation

In [290]:
def compute_rotation_2p(segment_point1, segment_point2, sub1, sub2, constraints, verbose):
    # Calculate vectors along the line segment
    vector_segment = segment_point2 - segment_point1
    vector_segment /= np.linalg.norm(vector_segment)

    def cost_angle_function(angle, X1, X2, axis, point, constraint):
        rotation = Rot.from_rotvec(angle * axis)
        cost = 0
        for i, j, mini, maxi in constraint:
            norm = np.linalg.norm(X1[i] - (np.dot(X2[j]- point, axis) * axis + rotation.apply((X2[j] - (X2[j] - point) @ axis * axis) - point) + point))
            if mini < norm < maxi:
                # cost += (norm - mini)*(maxi - norm) * 0
                pass
            else:
                cost += 50*(norm - mini)**2*(maxi - norm)**2
        return cost

    point_rotation = segment_point1 - np.dot(segment_point1, vector_segment) * vector_segment

    # res_angle = minimize_scalar(cost_angle_function, bounds=(0, 2*np.pi), args=(sub1.X, sub2.X, vector_segment, point_rotation, constraints), method='bounded', options={'xopt': 0.0000001})
    res_angle = minimize_scalar(cost_angle_function, bounds=(0, 2*np.pi), args=(sub1.X, sub2.X, vector_segment, point_rotation, constraints), method='bounded')
    if verbose >1:
        print(res_angle)
        print(f'minimizing_cost_function: {cost_angle_function(np.array(res_angle.x), sub1.X, sub2.X, vector_segment, point_rotation, constraints)}')
    rotation = Rot.from_rotvec(res_angle.x * vector_segment)
    
    return rotation, vector_segment, point_rotation

In [291]:
def compute_rotation_1p(fixed_point, sub1, sub2, constraints, verbose):

    def cost_angles_function(angles, X1, X2, point, constraint):
        angle_theta, angle_phi = angles
        rotation_theta = Rot.from_rotvec(angle_theta * np.array([0,0,1]))
        rotation_phi = Rot.from_rotvec(angle_phi * rotation_theta.apply(np.array([0,1,0])))
        rotation = np.dot(rotation_theta, rotation_phi)
        cost = 0
        for i, j, mini, maxi in constraint:
            norm = np.linalg.norm(X1[i] - (rotation.apply(X2[j] - point) + point))
            if mini < norm < maxi:
                cost -= 16 * ((norm - mini)*(maxi - norm)/(maxi - mini))**2
                pass
            else:
                cost += 50*(norm - mini)**2*(maxi - norm)**2
        return cost

    init_angle = np.array([0,0])
    # res_angle = minimize_scalar(cost_angle_function, bounds=(0, 2*np.pi), args=(sub1.X, sub2.X, vector_segment, point_rotation, constraints), method='bounded', options={'xopt': 0.0000001})
    res_angle = minimize(cost_angles_function, init_angle.flatten(), args=(sub1.X, sub2.X, fixed_point, constraints), method='Nelder-Mead')
    if verbose > 0:
        print(res_angle)
        print(f'minimizing_cost_function: {cost_angles_function(np.array(res_angle.x), sub1.X.copy(), sub2.X.copy(), fixed_point, constraints)}')
    angle_theta, angle_phi = res_angle.x
    rotation_theta = Rot.from_rotvec(angle_theta * np.array([0,0,1]))
    rotation_phi = Rot.from_rotvec(angle_phi * rotation_theta.apply(np.array([0,1,0])))
    rotation = np.dot(rotation_theta, rotation_phi)    
    return rotation

In [292]:
class sub_fc:

    def __init__(self, fc_subgraph) -> None:
        sub = nx.Graph()
        for node in fc_subgraph:
            sub.add_node(node)

        for (i,j), (mini, maxi) in dico_interval.items():
            if i in fc_subgraph and j in fc_subgraph:
                sub.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])

        self.sub = sub
        self.X = get_position_sub(sub).astype(np.float64)
        self.index_to_X, self.X_to_index = get_dico_sub(sub)
        self.dico_link = {}
        self.hierarchie = []
        self.position_atoms = {}
        self.origin_atom = {}


    def test_communs_positions(self, sub2):
        res = list(set(self.sub.nodes).intersection(set(sub2.sub.nodes)))
        return res

 
    def test_communs_constraints(self, sub2, dico_interval):
        nodes1 = list(self.sub.nodes)
        nodes2 = list(sub2.sub.nodes)
        nodes1 = [x for x in nodes1 if x not in nodes2]
        nodes2 = [x for x in nodes2 if x not in nodes1]
        list_constraint = []
        for i in nodes1:
            for j in nodes2:
                try:
                    if len(dico_interval[(i,j)]):

                        list_constraint.append((self.index_to_X[i],sub2.index_to_X[j], dico_interval[(i,j)][0], dico_interval[(i,j)][1]))
                except KeyError:
                    pass
        return list_constraint


    def apply_translation(self, translation):
        for i in range(len(self.X)):
            self.X[i] += translation


    def apply_rotation_point(self, rotation, point):
        for i in range(len(self.X)):
            self.X[i] = rotation.apply(self.X[i] - point) + point
        

    def apply_rotation_axis(self, rotation, axis, point):
        for i in range(len(self.X)):
            self.X[i] = np.dot(self.X[i], axis) * axis + rotation.apply((self.X[i] - self.X[i] @ axis * axis) - point) + point

    def visualisation (self, dico_interval):
        for i in self.sub.nodes:
            for j in self.sub.nodes:
                try:
                    if len(dico_interval[(i,j)]) > 1:
                        print(f"({i}, {j}) -- lenght: {np.linalg.norm(self.X[self.index_to_X[i]] - self.X[self.index_to_X[j]])}, interval: {dico_interval[(i,j)]}")
                except KeyError:
                    pass
        print()


    def update_position_3p(self, sub2, common_points, verbose):
        #translation
        translation = align_figures(self.X, sub2.X, self.index_to_X[common_points[0]], sub2.index_to_X[common_points[0]])
        sub2.apply_translation(translation)


        #rotation 1 
        if verbose >1:
            print(common_points[:2])
        first_rotation = calculate_rotation_matrix(
            sub2.X[sub2.index_to_X[common_points[0]]],
            sub2.X[sub2.index_to_X[common_points[1]]],
            self.X[self.index_to_X[common_points[1]]],
            verbose
            )
        sub2.apply_rotation_point(first_rotation, sub2.X[sub2.index_to_X[common_points[0]]])


        #rotation 2
        second_rotation, axis, point = calculate_sec_rotation_matrix(
            sub2.X[sub2.index_to_X[common_points[0]]],
            sub2.X[sub2.index_to_X[common_points[1]]],
            sub2.X[sub2.index_to_X[common_points[2]]],        
            self.X[self.index_to_X[common_points[2]]],
            verbose
            )
        sub2.apply_rotation_axis(
            second_rotation,
            axis,
            point)
        
        if verbose>0:
            print(f"results: {np.linalg.norm(sub2.X[sub2.index_to_X[common_points[0]]] - self.X[self.index_to_X[common_points[0]]])} --- {np.linalg.norm(sub2.X[sub2.index_to_X[common_points[1]]] - self.X[self.index_to_X[common_points[1]]])} ---{np.linalg.norm(sub2.X[sub2.index_to_X[common_points[2]]] - self.X[self.index_to_X[common_points[2]]])}")


    def update_position_2p(self, sub2, common_points, constraints, verbose):
            #translation
            translation = align_figures(self.X, sub2.X, self.index_to_X[common_points[0]], sub2.index_to_X[common_points[0]])
            sub2.apply_translation(translation)


            #rotation 1 
            first_rotation = calculate_rotation_matrix(
                sub2.X[sub2.index_to_X[common_points[0]]],
                sub2.X[sub2.index_to_X[common_points[1]]],
                self.X[self.index_to_X[common_points[1]]],
                verbose
                )
            sub2.apply_rotation_point(first_rotation, sub2.X[sub2.index_to_X[common_points[0]]])


            #rotation 2 (segment_point1, segment_point2, sub1, sub2, constraints, verbose)
            second_rotation, axis, point = compute_rotation_2p(
                sub2.X[sub2.index_to_X[common_points[0]]],
                sub2.X[sub2.index_to_X[common_points[1]]],
                self,
                sub2,
                constraints, 
                verbose
                )
            sub2.apply_rotation_axis(
                second_rotation,
                axis,
                point)
            
            if verbose>0:
                print(f"results: {np.linalg.norm(sub2.X[sub2.index_to_X[common_points[0]]] - self.X[self.index_to_X[common_points[0]]])} --- {np.linalg.norm(sub2.X[sub2.index_to_X[common_points[1]]] - self.X[self.index_to_X[common_points[1]]])}")
                for (i,j, min, max) in constraints:
                    print(f'constraint ({i}, {j}) --- norm: {np.linalg.norm(sub2.X[j] - self.X[i])}, constraint: {[min, max]}')


    def update_position_1p(self, sub2, common_points, constraints, verbose):
            #translation
            translation = align_figures(self.X, sub2.X, self.index_to_X[common_points[0]], sub2.index_to_X[common_points[0]])
            sub2.apply_translation(translation)


            #rotation 1
            #fixed_point, sub1, sub2, constraints, verbose
            rotation = compute_rotation_1p(
                sub2.X[sub2.index_to_X[common_points[0]]],
                self,
                sub2,
                constraints,
                verbose
                )

            sub2.apply_rotation_point(rotation, sub2.X[sub2.index_to_X[common_points[0]]])

            if verbose>0:
                print(f"results: {np.linalg.norm(sub2.X[sub2.index_to_X[common_points[0]]] - self.X[self.index_to_X[common_points[0]]])}")
                for (i,j, min, max) in constraints:
                    print(f'constraint ({i}, {j}) --- norm: {np.linalg.norm(sub2.X[j] - self.X[i])}, constraint: {[min, max]}')


    def reversing_position_3p(self, sub2, common_points):
        plan_vect_1 = sub2.X[sub2.index_to_X[common_points[0]]] - sub2.X[sub2.index_to_X[common_points[1]]]
        plan_vect_2 = sub2.X[sub2.index_to_X[common_points[0]]] - sub2.X[sub2.index_to_X[common_points[2]]]
        
        inversion_axis = np.cross(plan_vect_1, plan_vect_2)
        inversion_axis /= np.linalg.norm(inversion_axis)
        distance = np.dot(inversion_axis, sub2.X[sub2.index_to_X[common_points[0]]]) * inversion_axis

        for i in range(len(sub2.X)):
            comp_to_inverse = np.dot(sub2.X[i] - distance, inversion_axis)
            sub2.X[i] -= 2 * inversion_axis * comp_to_inverse


    def reversing_position_1_2p(self, sub2):
        for i in range(len(sub2.X)):
            sub2.X[i][0] = -sub2.X[i][0]


    def test_constraint_merge(self, sub2, dico_interval, verbose):
        nodes1 = list(self.sub.nodes)
        nodes2 = list(sub2.sub.nodes)
        
        for i in nodes1 + nodes2:
            for j in nodes1 + nodes2:
                try :
                    if dico_interval[(i,j)][0] < \
                        np.linalg.norm(self.X[self.index_to_X[i]] - sub2.X[sub2.index_to_X[j]])\
                        < dico_interval[(i,j)][1]:
                        continue
                    else:
                        if verbose > 0:
                            print(f"atomes ({i},{j}): {dico_interval[(i,j)]} --- {np.linalg.norm(self.X[self.index_to_X[i]] - sub2.X[sub2.index_to_X[j]])}")
                        return False
                except KeyError:
                    continue
        return True


    def test_constraint(self, dico_interval, approx=False):
        nodes = list(self.sub.nodes)
        for i in nodes:
            for j in nodes[i+1:]:
                try :
                    if dico_interval[(i,j)][0] < \
                        np.linalg.norm(self.X[self.index_to_X[i]] - self.X[self.index_to_X[j]])\
                        < dico_interval[(i,j)][1]:
                        continue
                    else:
                        print(f"atomes ({i},{j}): {dico_interval[(i,j)]} --- {np.linalg.norm(self.X[self.index_to_X[i]] - self.X[self.index_to_X[j]])}")
                        if not approx:
                            return False
                except KeyError:
                    continue
        return True

    def merge_X(self, sub2, sub):
        X = []
        for node in sub.nodes:
            pos1, pos2 = [], []
            try:
                pos1 = np.array(self.X[self.index_to_X[node]])
            except KeyError:
                pass
            try:
                pos2 = np.array(sub2.X[sub2.index_to_X[node]])
            except KeyError:
                pass
            if len(pos1) == 0:
                X.append(pos2)
            elif len(pos2) == 0:
                X.append(pos1)
            else:
                X.append((pos1 + pos2) / 2)
        index_to_X, X_to_index = get_dico_sub(sub)
        return X, index_to_X, X_to_index


    def merge(self, sub2, dico_interval, add_constraint = True):
        sub = nx.Graph()
        for node in self.sub.nodes:
            sub.add_node(node)
        for node in sub2.sub.nodes:
            if node not in self.sub.nodes:
                sub.add_node(node)

        for (i,j), (mini, maxi) in dico_interval.items():
            if i in sub.nodes and j in sub.nodes:
                sub.add_edges_from([(i,j,{"mini":mini, "maxi":maxi})])

        self.sub = sub
        self.X, self.index_to_X, self.X_to_index = self.merge_X(sub2, sub)

        if add_constraint:
            delta = 0.01
            for i in sub.nodes:
                for j in sub.nodes:
                    if i==j:
                        continue
                    try:
                        dico_interval[(i,j)]
                    except KeyError:
                        new_contraint = np.linalg.norm(self.X[self.index_to_X[i]] - self.X[self.index_to_X[j]])
                        dico_interval[(i,j)] = [new_contraint - delta, new_contraint + delta]
    
    def compute_single_atom(self, atom_to_add, dico_interval, verbose=0, approx=False):
        list_index= []
        update = False
        for i in self.sub.nodes:
            try:
                if dico_interval[(i, atom_to_add)]:
                    list_index.append(i)
            except KeyError:
                continue
        
        if verbose>1:
            print(f"adding {atom_to_add}: constraints: {list_index}")

        def cost_distance_function(position, sub, list_index, atom_to_add, constraint):
            cost = 0
            for i in list_index:
                mini, maxi = constraint[(i, atom_to_add)]
                norm = np.linalg.norm(sub.X[sub.index_to_X[i]] - position)
                if mini < norm < maxi:
                    # cost += (norm - mini)*(maxi - norm) * 0
                    pass
                else:
                    cost += 50*(norm - mini)**2*(maxi - norm)**2
            return cost

        init_position = np.array([0,0,0])
        res_position = minimize(cost_distance_function, init_position, args=(self, list_index, atom_to_add, dico_interval), method='Nelder-Mead')
        if verbose >1:
            print(res_position)
            print(f'minimizing_cost_function: {cost_distance_function(np.array(res_position.x), self, list_index, atom_to_add, dico_interval)}')
        
        if (res_position.fun < 0.01) or approx:
            self.add_atom(atom_to_add, res_position.x, dico_interval)
            update = True

        return update
    
    def add_atom(self, atom_index, position, dico_interval):
        n = len(self.sub.nodes)
        self.sub.nodes = list(self.sub.nodes) + [atom_index]
        try:
            self.X.append(position)
        except AttributeError:
            self.X = np.concatenate((self.X, position.reshape(1, -1)), axis=0)
        self.index_to_X[atom_index] = n
        self.X_to_index[n] = atom_index

    
    def update_atom(self, point, position):
        self.X[self.index_to_X[point]] = np.array(position)

    def create_hierarchie(self, connection_atoms):
        hierarchie = [list(self.sub.nodes)]
        tot_hierarchie = list(self.sub.nodes)
        n = len(hierarchie[0])
        temp_hier=[]
        while n + len(temp_hier) < N_ATOMS:
            for i in hierarchie[-1]:
                for atom in connection_atoms[i]:
                    if atom in tot_hierarchie:
                        continue
                    else:
                        temp_hier.append(atom)
            set_temp = [elem for elem in temp_hier if temp_hier.count(elem)>1]
            temp_hier = [elem for elem in temp_hier if temp_hier.count(elem)==1]
            set_temp = list(set(set_temp))
            n += len(set_temp)
            hierarchie.append(set_temp)
            tot_hierarchie += set_temp
        hierarchie.append(temp_hier)
        self.hierarchie = hierarchie
    
    def traduct_hierarchie(self, connection_atoms):
        for hier in self.hierarchie[2:]:
            for atom in hier:
                to_up = set(connection_atoms[atom]) & set(self.hierarchie[0])
                for i in list(to_up):
                    self.position_atoms[i] = [(self.X[self.index_to_X[i]], self.X[self.index_to_X[i]] + np.ones((3,))*0.0001)]







In [293]:
cliques = nx.find_cliques(G)
list_sub = []
for fc_subgraph in cliques:
    if len(fc_subgraph)<4:
        continue

    list_sub.append(sub_fc(fc_subgraph))

In [294]:
for i in list_sub:
    print(i.sub.nodes, end=' --- ')
    i.test_constraint(dico_interval)


[0, 1, 2, 3, 4] --- [1, 5, 2, 3] --- [3, 2, 16, 17] --- [3, 2, 16, 5] --- [3, 2, 17, 4] --- [5, 8, 6, 7] --- [8, 13, 16, 15] --- [14, 13, 16, 17, 15] --- [18, 20, 21, 10] --- [18, 20, 21, 19, 22, 24, 25, 26, 23] --- [28, 30, 32, 27, 29, 31] --- [29, 31, 27, 33] --- [29, 31, 38, 33] --- [31, 33, 34, 37, 38] --- [33, 35, 34, 36, 37, 38] --- [34, 56, 58, 59, 54] --- [39, 41, 40, 43, 42, 44] --- [39, 41, 40, 43, 19] --- [45, 46, 47, 48, 49] --- [50, 51, 52, 53] --- [50, 51, 52, 54] --- [54, 55, 56, 57, 58, 59, 60, 61, 62] --- 

In [295]:
def test_and_merge_3p(i,j,list_sub, dico_interval, verbose=0, approx=False):
    """ verbose has 3 stades: 0 no verbose, 1 mini verbose for each merging, 2 total verbose"""
    sub1 = list_sub[i]
    sub2 = list_sub[j]
    commun_points = sub1.test_communs_positions(sub2)
    if len(commun_points) < 3:
        return False
    else:
        if verbose > 0:
            print()
            print(f"3p_test i = {i}, j = {j}: n_communs: {len(commun_points)} --- {sub1.sub.nodes} --- {sub2.sub.nodes}")
            if verbose > 1:
                sub1.visualisation(dico_interval)
                sub2.visualisation(dico_interval)
        sub1.update_position_3p(sub2, commun_points, verbose)


        if verbose > 1:
            for j in list(sub1.sub.nodes) + list(sub1.sub.nodes):
                print(f'atome {j}')
                try: 
                    print(sub1.X[sub1.index_to_X[j]])
                except KeyError:
                    pass
                try:
                    print(sub2.X[sub2.index_to_X[j]])
                except KeyError:
                    pass

        if approx:
            sub1.merge(sub2, dico_interval)
            return True
        if sub1.test_constraint_merge(sub2, dico_interval, verbose):
            sub1.merge(sub2, dico_interval)
            return True
        else:
            sub1.reversing_position_3p(sub2, commun_points)
            if sub1.test_constraint_merge(sub2, dico_interval, verbose):
                sub1.merge(sub2, dico_interval)
        return False

In [296]:
def merging_3p(list_sub, verbose, approx=False):
    n = len(list_sub)
    merge_succeed, has_merged = True, False
    while merge_succeed:
        merge_succeed = False
        i, j = 0, 1
        while i < len(list_sub):
            while j < len(list_sub):
                if test_and_merge_3p(i,j,list_sub, dico_interval, verbose, approx=approx):
                    list_sub.pop(j)
                    merge_succeed, has_merged = True, True
                else:
                    j +=1
            i+=1
            j = i + 1
    print(f"{n} => {len(list_sub)}")
    return list_sub, has_merged

# list_sub, _ = merging_3p(list_sub, verbose=0)[0]
# print(list_sub)

In [297]:
# visualize_step(list_sub=list_sub, dico_interval=dico_interval)

In [298]:
def test_and_merge_2p(i,j,list_sub, dico_interval, verbose=0, approx=False):
    """ verbose has 3 stades: 0 no verbose, 1 mini verbose for each merging, 2 total verbose"""
    sub1 = list_sub[i]
    sub2 = list_sub[j]
    commun_points = sub1.test_communs_positions(sub2)
    if len(commun_points) < 2:
        return False
    commun_constraints = sub1.test_communs_constraints(sub2, dico_interval)
    if len(commun_constraints)<1:
        return False
    if verbose > 0:
        print()
        print(commun_constraints)
        print(f"2p_test i = {i}, j = {j}: n_communs: {len(commun_points)} --- n_communs_constraints: {len(commun_constraints)} --- {sub1.sub.nodes} --- {sub2.sub.nodes}")
        if verbose > 1:
            sub1.visualisation(dico_interval)
            sub2.visualisation(dico_interval)
    
    sub1.update_position_2p(sub2, commun_points, commun_constraints, verbose)

    if verbose > 1:
        for j in list(sub1.sub.nodes) + list(sub1.sub.nodes):
            print(f'atome {j}')
            try: 
                print(sub1.X[sub1.index_to_X[j]])
            except KeyError:
                pass
            try:
                print(sub2.X[sub2.index_to_X[j]])
            except KeyError:
                pass

    if approx:
        sub1.merge(sub2, dico_interval)
        return True
    if sub1.test_constraint_merge(sub2, dico_interval, verbose):
        sub1.merge(sub2, dico_interval)
        return True
    else:
        sub1.reversing_position_1_2p(sub2)
        sub1.update_position_2p(sub2, commun_points, commun_constraints, verbose)

        if verbose > 1:
            for j in list(sub1.sub.nodes) + list(sub1.sub.nodes):
                print(f'atome {j}')
                try: 
                    print(sub1.X[sub1.index_to_X[j]])
                except KeyError:
                    pass
                try:
                    print(sub2.X[sub2.index_to_X[j]])
                except KeyError:
                    pass

        if sub1.test_constraint_merge(sub2, dico_interval, verbose):
            sub1.merge(sub2, dico_interval)
            return True
    return False

In [299]:
def merging_2p(list_sub, verbose, approx=False):
    n = len(list_sub)
    merge_succeed, has_merged = True, False
    while merge_succeed:
        merge_succeed = False
        i, j = 0, 1
        while i < len(list_sub):
            while j < len(list_sub):
                if test_and_merge_2p(i,j,list_sub, dico_interval, verbose, approx=approx):
                    list_sub.pop(j)
                    merge_succeed, has_merged = True, True
                else:
                    j +=1
            i+=1
            j = i + 1
    print(f"{n} => {len(list_sub)}")
    return list_sub, has_merged

In [300]:
# list_sub_2p = copy.deepcopy(list_sub)

# list_sub_2p, _ = merging_2p(list_sub_2p, verbose=2)

In [301]:
# visualize_step(list_sub=list_sub_2p, dico_interval=dico_interval)

In [302]:
def test_and_merge_1p(i,j,list_sub, dico_interval, verbose=0, approx=False):
    """ verbose has 3 stades: 0 no verbose, 1 mini verbose for each merging, 2 total verbose"""
    sub1 = list_sub[i]
    sub2 = list_sub[j]
    commun_points = sub1.test_communs_positions(sub2)
    if len(commun_points) < 1:
        return False
    commun_constraints = sub1.test_communs_constraints(sub2, dico_interval)
    if len(commun_constraints)<1:
        return False
    if verbose > 0:
        print()
        print(commun_constraints)
        print(f"1p_test i = {i}, j = {j}: n_communs: {len(commun_points)} --- n_communs_constraints: {len(commun_constraints)} --- {sub1.sub.nodes} --- {sub2.sub.nodes}")
        if verbose > 1:
            sub1.visualisation(dico_interval)
            sub2.visualisation(dico_interval)
    
    sub1.update_position_1p(sub2, commun_points, commun_constraints, verbose)

    if verbose > 1:
        for j in list(sub1.sub.nodes) + list(sub1.sub.nodes):
            print(f'atome {j}')
            try: 
                print(sub1.X[sub1.index_to_X[j]])
            except KeyError:
                pass
            try:
                print(sub2.X[sub2.index_to_X[j]])
            except KeyError:
                pass

    if approx:
        sub1.merge(sub2, dico_interval)
        return True
    if sub1.test_constraint_merge(sub2, dico_interval, verbose):
        sub1.merge(sub2, dico_interval)
        return True
    else:
        sub1.reversing_position_1_2p(sub2)
        sub1.update_position_1p(sub2, commun_points, commun_constraints, verbose)

        if verbose > 1:
            for j in list(sub1.sub.nodes) + list(sub1.sub.nodes):
                print(f'atome {j}')
                try: 
                    print(sub1.X[sub1.index_to_X[j]])
                except KeyError:
                    pass
                try:
                    print(sub2.X[sub2.index_to_X[j]])
                except KeyError:
                    pass

        if sub1.test_constraint_merge(sub2, dico_interval, verbose):
            sub1.merge(sub2, dico_interval)
            return True
    return False

In [303]:
def merging_1p(list_sub, verbose, approx=False):
    n = len(list_sub)
    merge_succeed, has_merged = True, False
    while merge_succeed:
        merge_succeed = False
        i, j = 0, 1
        while i < len(list_sub) - 1:
            while j < len(list_sub):
                if test_and_merge_1p(i,j,list_sub, dico_interval, verbose, approx=approx):
                    list_sub.pop(j)
                    merge_succeed, has_merged = True, True
                else:
                    j +=1
            i+=1
            j = i + 1
    print(f"{n} => {len(list_sub)}")
    return list_sub, has_merged

In [304]:
# list_sub_1p = copy.deepcopy(list_sub_2p)

# list_sub_1p, _ = merging_1p(list_sub_1p, verbose=0)

In [305]:
# visualize_step(list_sub=list_sub_1p, dico_interval=dico_interval)

In [306]:
def link_atoms_to_sub(list_sub, dico_interval, verbose=0):
    for sub in list_sub:
        sub.dico_link = {}
        for atom in range(N_ATOMS):
            if atom in sub.sub.nodes:
                continue
            for atom_2 in sub.sub.nodes:
                try:
                    dico_interval[(atom, atom_2)]
                    try:
                        if sub.dico_link[atom]:
                            sub.dico_link[atom] = sub.dico_link[atom]+1
                    except KeyError:
                        sub.dico_link[atom] = 1
                except KeyError:
                    continue
        if verbose:
            print(f"for sub: {sub.sub.nodes} --- {sub.dico_link}")

In [307]:
def add_linked_atoms(list_sub, dico_interval, verbose=0, approx=False):
    update_global = False
    link_atoms_to_sub(list_sub, dico_interval, verbose=verbose)
    for sub in list_sub:
        for atom_to_add in sub.dico_link:
            if sub.dico_link[atom_to_add] < 3:
                continue

            update = sub.compute_single_atom(atom_to_add, dico_interval, verbose=verbose, approx=approx)
            if update:
                update_global = True
            
    return list_sub, update_global

In [308]:
def add_single_atomes(list_sub, dico_interval, verbose=0, approx=False, merge_all = True):
    update, global_update = True, False
    while update:
        update = False
        list_sub, update = add_linked_atoms(list_sub, dico_interval, verbose=verbose, approx=approx)
        if merge_all and not update:
            list_sub, update = merging_all(list_sub, dico_interval, verbose=verbose, update=True, approx=approx)
        global_update = global_update or update
    return list_sub, global_update


In [309]:
def merging_all(list_sub, dico_interval, verbose=0, approx=False, update=False, merging_1_atom=False):
    n_init = len(list_sub)
    has_merged = True
    has_merged_1a = False
    while has_merged:
        has_merged = False
        
        list_sub, has_merged_3p = merging_3p(list_sub, verbose, approx=approx)
    
        if merging_1_atom:
            list_sub, has_merged_1a = add_single_atomes(list_sub, dico_interval, verbose=0, approx=approx, merge_all=False)


        list_sub, has_merged_2p = merging_2p(list_sub, verbose, approx=approx)

        if not has_merged_2p:
            list_sub, has_merged_1p = merging_1p(list_sub, verbose=verbose, approx=approx)

        visualize_step(list_sub=list_sub, dico_interval=dico_interval)
        has_merged = has_merged_3p or has_merged_2p or has_merged_1p or has_merged_1a
    print(f"evolution global: {n_init} => {len(list_sub)}")
    if update:
        return list_sub, n_init != len(list_sub)
    return list_sub

In [310]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

## Merge part

In [ ]:
list_sub = merging_all(list_sub, dico_interval, verbose=1, merging_1_atom=False)

In [ ]:
visualize_step(list_sub=list_sub, dico_interval=dico_interval)

In [ ]:
list_sub = merging_all(list_sub, dico_interval, verbose=0, approx=False)

In [ ]:
link_atoms_to_sub(list_sub, dico_interval, verbose=2)

In [ ]:
visualize_step(list_sub=list_sub, dico_interval=dico_interval)

if len(list_sub)>1:
    list_sub, _ = add_single_atomes(list_sub, dico_interval, verbose=0, approx=False)
    visualize_step(list_sub=list_sub, dico_interval=dico_interval)


In [ ]:
if len(list_sub)>1:
    list_sub = merging_all(list_sub, dico_interval, verbose=0, approx=False)
if len(list_sub[0].sub.nodes)<N_ATOMS:
    list_sub, _ = add_single_atomes(list_sub, dico_interval, verbose=0, approx=False)

In [ ]:
visualize_step(list_sub=list_sub, dico_interval=dico_interval, vision=False)
final_sub = list_sub[0]

In [ ]:
if True:
    # Create a 3D scatter plot
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')

    connection_between_atoms = {}
    # Extract x, y, and z coordinates
    x,y,z = [], [], []
    for i in range(N_ATOMS):
        x.append(final_sub.X[final_sub.index_to_X[i]][0])
        y.append(final_sub.X[final_sub.index_to_X[i]][1])
        z.append(final_sub.X[final_sub.index_to_X[i]][2])

    # Plot the atoms
    ax.scatter(x, y, z, c='b', marker='o', s=10)

    # Set labels for the axes
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')

    # Plot the lines between connected atoms
    good, bad = 0,0
    for i,j in dico_init_inter:
        if i < j:
            continue 

        x_link_1, y_link_1, z_link_1 = final_sub.X[final_sub.index_to_X[i]]
        x_link_2, y_link_2, z_link_2 = final_sub.X[final_sub.index_to_X[j]]

        mini, maxi = dico_init_inter[(i,j)]

        color = (mini < ((x_link_1 - x_link_2)**2 + (y_link_1 - y_link_2)**2 + (z_link_1 - z_link_2)**2)**.5 < maxi )

        try:
            if connection_between_atoms[i]:
                connection_between_atoms[i][color].append(j)
        except KeyError:
            connection_between_atoms[i] = [[],[]]
            connection_between_atoms[i][color].append(j)

        try:
            if connection_between_atoms[j]:
                connection_between_atoms[j][color].append(i)
        except KeyError:
            connection_between_atoms[j] = [[],[]]
            connection_between_atoms[j][color].append(i)



        if color:
            ax.plot((x_link_1, x_link_2), (y_link_1, y_link_2), (z_link_1, z_link_2), c='g')
            good +=1
        else:
            ax.plot((x_link_1, x_link_2), (y_link_1, y_link_2), (z_link_1, z_link_2), c='r')
            bad +=1

    print(good, bad, good/(bad+good))

    # Show the 3D plot
    plt.show()

In [ ]:
pos_final = [0 for i in range(3*N_ATOMS)]

for i in range(N_ATOMS):
    pos_final[3*i] = final_sub.X[final_sub.index_to_X[i]][0]
    pos_final[3*i+1] = final_sub.X[final_sub.index_to_X[i]][1]
    pos_final[3*i+2] = final_sub.X[final_sub.index_to_X[i]][2]

print(pos_final)

In [ ]:
%store pos_final
break


## Marco part 2 Fine tunning 

In [ ]:
list_of_ratio = [0 for i in range(N_ATOMS)]
for i in connection_between_atoms:
    list_of_ratio[i] = len(connection_between_atoms[i][1])/(len(connection_between_atoms[i][0])+len(connection_between_atoms[i][1]))

In [ ]:
def optimize_a_point(sub, point_index, dico_interval, connection_between_atoms, list_of_ratio):
    list_of_linked = connection_between_atoms[point_index][0] + connection_between_atoms[point_index][1]

    def cost_distance_function(position, sub, list_index, atom_to_add, constraint, list_of_ratio):
        cost = 0
        for i in list_index:
            mini, maxi = constraint[(i, atom_to_add)]
            norm = np.linalg.norm(sub.X[sub.index_to_X[i]] - position)
            if mini < norm < maxi:
                #cost -= 50*(norm - mini)*(maxi - norm) * list_of_ratio[i]
                pass
            else:
                # cost += 50*(norm - mini)**2*(maxi - norm)**2 * list_of_ratio[i]
                cost += (norm - mini)**2*(maxi - norm)**2 * list_of_ratio[i]
                pass
        return cost

    init_position = sub.X[sub.index_to_X[point_index]] + np.random.uniform(low=-0.05, high=0.05, size=3)
    res_position = minimize(cost_distance_function, init_position, args=(sub, list_of_linked, point_index, dico_interval, list_of_ratio), method='Nelder-Mead')
    
    if (res_position.fun < 0.01):
        sub.update_atom(point_index, res_position.x)
        return True
    return False

In [ ]:
def update_ratio_point(sub, i, connection, dico_init_inter, list_of_ratio):
    for before_value in range(2):
        for j in connection[i][before_value]:
            x_link_1, y_link_1, z_link_1 = final_sub.X[final_sub.index_to_X[i]]
            x_link_2, y_link_2, z_link_2 = final_sub.X[final_sub.index_to_X[j]]

            mini, maxi = dico_init_inter[(i,j)]

            color = (mini < ((x_link_1 - x_link_2)**2 + (y_link_1 - y_link_2)**2 + (z_link_1 - z_link_2)**2)**.5 < maxi )

            if color != before_value:
                connection_between_atoms[i][color].append(j)
                connection_between_atoms[j][color].append(i)

                connection_between_atoms[i][1-color].remove(j)
                connection_between_atoms[j][1-color].remove(i)
    
    for value in range(2):
        for j in connection[i][value]:
            list_of_ratio[j] = len(connection_between_atoms[j][1])/(len(connection_between_atoms[j][0])+len(connection_between_atoms[j][1]))
    
    list_of_ratio[i] = len(connection_between_atoms[i][1])/(len(connection_between_atoms[i][0])+len(connection_between_atoms[i][1]))
    



In [ ]:
def optimise_the_graph(sub, list_of_ratio, connection, dico_init_inter):
    print(f"initial_loss = {N_ATOMS - np.sum(list_of_ratio)}")
    while np.sum(list_of_ratio) < N_ATOMS - 1:
        i = np.random.randint(N_ATOMS)
        ratio = list_of_ratio[i]
        if ratio < 1:
            update = optimize_a_point(sub, i, dico_init_inter, connection, list_of_ratio)
            if update:
                update_ratio_point(sub, i, connection, dico_init_inter, list_of_ratio)
                print(f"loss = {N_ATOMS - np.sum(list_of_ratio)} --- updated {i}")



In [ ]:
def optimise_graph(sub, dico_interval):

    def cost_distance_function(position, sub, constraint):
        cost = 0
        for i in range(N_ATOMS):
            for j in range(i+1, N_ATOMS):
                try:
                    mini, maxi = constraint[(i, j)]
                except KeyError:
                    continue
                norm = np.linalg.norm(np.array(position[3*j:3*j+3]) - np.array(position[3*i:3*i+3]))
                if mini < norm < maxi:
                    cost -= (norm - mini)*(maxi - norm)
                    pass
                else:
                    cost += 50*(norm - mini)**2*(maxi - norm)**2
        return cost

    init_positions = []
    for i in sub.X:
        init_positions.append(i[0])
        init_positions.append(i[1])
        init_positions.append(i[2])
    init_positions = np.array(init_positions)
    res_position = minimize(cost_distance_function, init_positions, args=(sub,dico_interval), method='BFGS')
    
    print(res_position)


In [ ]:
if 1:
    optimise_the_graph(final_sub, list_of_ratio, connection_between_atoms, dico_init_inter)
else:
    optimise_graph(final_sub, dico_init_inter)

In [ ]:
# Create a 3D scatter plot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

connection_between_atoms = {}
# Extract x, y, and z coordinates
x,y,z = [], [], []
for i in range(N_ATOMS):
    x.append(final_sub.X[i][0])
    y.append(final_sub.X[i][1])
    z.append(final_sub.X[i][2])

# Plot the atoms
ax.scatter(x, y, z, c='b', marker='o', s=10)

# Set labels for the axes
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Plot the lines between connected atoms
good, bad = 0,0
for i,j in dico_init_inter:
    if i < j:
        continue 

    x_link_1, y_link_1, z_link_1 = final_sub.X[final_sub.index_to_X[i]]
    x_link_2, y_link_2, z_link_2 = final_sub.X[final_sub.index_to_X[j]]

    mini, maxi = dico_init_inter[(i,j)]

    color = (mini < ((x_link_1 - x_link_2)**2 + (y_link_1 - y_link_2)**2 + (z_link_1 - z_link_2)**2)**.5 < maxi )

    try:
        if connection_between_atoms[i]:
            connection_between_atoms[i][color].append(j)
    except KeyError:
        connection_between_atoms[i] = [[],[]]
        connection_between_atoms[i][color].append(j)

    try:
        if connection_between_atoms[j]:
            connection_between_atoms[j][color].append(i)
    except KeyError:
        connection_between_atoms[j] = [[],[]]
        connection_between_atoms[j][color].append(i)



    if color:
        ax.plot((x_link_1, x_link_2), (y_link_1, y_link_2), (z_link_1, z_link_2), c='g')
        good +=1
    else:
        ax.plot((x_link_1, x_link_2), (y_link_1, y_link_2), (z_link_1, z_link_2), c='r')
        bad +=1

print(good, bad, good/(bad+good))

# Show the 3D plot
plt.show()

In [ ]:
break

## Part 3 Noyau dur

In [ ]:
list_of_ratio = [0 for i in range(N_ATOMS)]
for i in connection_between_atoms:
    list_of_ratio[i] = len(connection_between_atoms[i][1])/(len(connection_between_atoms[i][0])+len(connection_between_atoms[i][1]))

In [ ]:
ker = []
for i in range(len(list_of_ratio)):
    if list_of_ratio[i] == 1:
        keep = True
        for j in connection_between_atoms:
            if list_of_ratio[j] == 1:
                

## 4 A la zeub

In [311]:
# list_sub = merging_all(list_sub, dico_interval, verbose=0, merging_1_atom=False)
#visualize_step(list_sub=list_sub, dico_interval=dico_interval)

In [312]:
index_sub = np.argmax(np.array([len(i.sub.nodes) for i in list_sub]))
subf = list_sub[index_sub]
print(subf.sub.nodes)

[18, 20, 21, 19, 22, 24, 25, 26, 23]


In [313]:
connection_atoms = {}
for i,j in dico_init_inter:
    if i>j:
        continue
    try:
        connection_atoms[i].append(j)
    except KeyError:
        connection_atoms[i] = [j]
    try:
        connection_atoms[j].append(i)
    except KeyError:
        connection_atoms[j] = [i]

In [314]:
subf.create_hierarchie(connection_atoms)
for i in subf.hierarchie:
    print(i)

subf.traduct_hierarchie(connection_atoms)

[18, 20, 21, 19, 22, 24, 25, 26, 23]
[2, 10, 44, 47, 49, 27]
[39, 40, 41, 43, 45, 46, 16, 48]
[33, 3, 4, 5, 42, 13, 17, 50, 28]
[0, 1, 32, 35, 8, 14, 15, 53, 29, 30, 31]
[34, 36, 37, 38, 6, 7, 12, 51, 52, 58]
[9, 11, 54, 56, 59]
[55, 57, 60, 61, 62]


In [315]:
def get_f_ctc(list_cst, node_add, subf, dico_init_inter):
    li_ctc = []
    li_constrains = []
    li_f = []
    atom_0 = subf.X[subf.index_to_X[list_cst[0]]]
    mini, maxi = dico_init_inter[(list_cst[0], node_add)]

    for atom in list_cst[1:]:
        low, up = dico_init_inter[(atom, node_add)]
        li_constrains.append([low,up])
        center = subf.X[subf.index_to_X[atom]]
        f = Function("alpha[3]", f"sqrt(sqr(alpha[0] * sin(alpha[2]) * cos(alpha[1]) + {atom_0[0]} - {center[0]})+sqr(alpha[0] * sin(alpha[2]) * sin(alpha[1])+ {atom_0[1]}-{center[1]})+sqr(alpha[0] * cos(alpha[2])+ {atom_0[2]}-{center[2]}))")
        li_f.append(f)
        contractor = CtcFunction(f, Interval(low,up))
        li_ctc.append(contractor)
    return li_ctc,li_f, li_constrains, mini, maxi, atom_0
        

In [316]:
def get_f_ctc_2(list_cst, node_add, combination, subf=subf, dico_init_inter=dico_init_inter, atom = None):
    li_ctc = []
    li_constrains = []
    li_f = []
    atom = list_cst[0]
    atom_0, max_0 = combination[0]
    delta_0 = np.linalg.norm(atom_0 - max_0)
    mini, maxi = dico_init_inter[(atom, node_add)]

    for index in range(1, len(combination)):
        low, up = dico_init_inter[(list_cst[index], node_add)]
        li_constrains.append([low,up])
        center, maximum = combination[index]
        delta = np.linalg.norm(maximum - center)
        f = Function("alpha[3]", f"sqrt(sqr(alpha[0] * sin(alpha[2]) * cos(alpha[1]) + {atom_0[0]} - {center[0]})+sqr(alpha[0] * sin(alpha[2]) * sin(alpha[1])+ {atom_0[1]}-{center[1]})+sqr(alpha[0] * cos(alpha[2])+ {atom_0[2]}-{center[2]}))")
        li_f.append(f)
        contractor = CtcFunction(f, Interval(low - delta_0 - delta,up + delta + delta_0))
        li_ctc.append(contractor)

    return li_ctc,li_f, li_constrains, mini, maxi, atom_0

In [317]:
node_to_add = 47
li_ctc,li_f, li_constrains, min, max, point = get_f_ctc(list(set(subf.hierarchie[0]).intersection(connection_atoms[node_to_add])), node_to_add, subf, dico_init_inter)
print(li_ctc,li_f, li_constrains, sep='\n')


[_f_108:(alpha[3])->sqrt(((((((alpha(1)*sin(alpha(3)))*cos(alpha(2)))+-1.21334)--1.35793)^2+((((alpha(1)*sin(alpha(3)))*sin(alpha(2)))+0.109014)-0.118084)^2)+(((alpha(1)*cos(alpha(3)))+-1.18948)-0.554672)^2))]
[[2.585608032939251, 2.637847815928732]]


In [318]:
if True:

    def contract(x,li_ctc):
        for ctc in li_ctc:
            ctc.contract(x)
    
    def empty(X):
        for i in range(2):
            if X[i].is_empty():
                return True
        return False
    def C_in(X, li_f, li_constrains):
        if (empty(X)):
            return False
        for i in range(len(li_f)):
            eval = li_f[i].eval(X)
            low, up = li_constrains[i]
            if ((eval.lb() < low) or (eval.ub()>up)):
                #in_it = False
                return False
        return True
    def C_out(X,li_f, li_constrains):
        out_it = False
        for i in range(len(li_f)):
            eval = li_f[i].eval(X)
            low, up = li_constrains[i]
            if ((eval.lb()> up) or (eval.ub() < low)):
                out_it = True
        return out_it

    def get_largest(X, thau):
        max_index = 3
        for i in range(2):
            if (thau[i] < X[i].diam()):
                max_index = i
        return max_index

In [319]:
def get_possibilities_from_ker(li_ctc,li_f, li_constrains, mini, maxi, point):
    tau = ((maxi-mini),0.1, 0.1)
    stack = [IntervalVector([[mini, maxi], [-np.pi, np.pi], [0, np.pi]])]
    stack_acc = []
    stack_rej = []
    stack_unc = []
    count = 0
    while len(stack) !=0:
        x = stack.pop()
        count +=1
        contract(x, li_ctc)
        if not empty(x):
            if (C_in(x,li_f, li_constrains)):
                stack_acc.append(x)
            else:
                if C_out(x, li_f, li_constrains):
                    stack_rej.append(x)
                else:
                    larger_index = get_largest(x, tau)
                    if larger_index<3:
                        X_left, X_right  = x.bisect(larger_index)
                        if not (empty(X_left)):
                            stack.append(X_left)
                        if not (empty(X_right)):
                            stack.append(X_right)
                    else:
                        stack_unc.append(x)
    return stack_acc + stack_unc, point

In [320]:
def update_position_atoms_format2(stacks, point, i, subf = subf):
    n = len(subf.position_atoms[i])
    subf.position_atoms[i] += [None for _ in range(len(stacks))]
    for j, stack in enumerate(stacks):
        r = sum(stack[0])/2
        theta = sum(stack[1])/2
        phi = sum(stack[2])/2
        subf.position_atoms[i][j + n] = (
            np.array([
                r * math.sin(phi) * math.cos(theta),
                r * math.sin(phi) * math.sin(theta),
                r * math.cos(phi)
            ]) + point,
            np.array([
                stack[0][1] * math.sin(stack[2][1]) * math.cos(stack[1][1]),
                stack[0][1] * math.sin(stack[2][1]) * math.sin(stack[1][1]),
                stack[0][1] * math.cos(stack[2][1])
            ]) + point,
        )

In [321]:
def get_possibilities_from_inter(li_ctc,li_f, li_constrains, mini, maxi, point, i):
    tau = ((maxi-mini), 0.1, 0.1)
    stack = [IntervalVector([[mini, maxi], [-np.pi, np.pi], [0, np.pi]])]
    stack_acc = []
    stack_rej = []
    stack_unc = []
    count = 0
    while len(stack) !=0:
        x = stack.pop()
        count += 1
        contract(x, li_ctc)
        if not empty(x):
            if (C_in(x,li_f, li_constrains)):
                stack_acc.append(x)
            else:
                if C_out(x, li_f, li_constrains):
                    stack_rej.append(x)
                else:
                    larger_index = get_largest(x,tau)
                    if larger_index<3:
                        X_left, X_right  = x.bisect(larger_index)
                        if not (empty(X_left)):
                            stack.append(X_left)
                        if not (empty(X_right)):
                            stack.append(X_right)
                    else:
                        stack_unc.append(x)
    return (stack_acc + stack_unc, point, i)

In [322]:
def initiate_atoms(sub, dico_init_inter, connection_atoms):
    for i in sub.hierarchie[1]:
        cst = list(set(subf.hierarchie[0]).intersection(connection_atoms[i]))
        li_ctc,li_f, li_constrains, mini, maxi, point = get_f_ctc(cst, i, subf, dico_init_inter)
        sub.position_atoms[i], sub.origin_atom[i] = get_possibilities_from_ker(li_ctc,li_f, li_constrains, mini, maxi, point)
        print(sub.position_atoms[i])

In [323]:
def test_stack(args, subf=subf, dico_init_inter=dico_init_inter, connection_atoms=connection_atoms):
    combination, index = args
    for i, r in enumerate(index):
        for j, k in enumerate(set(index[:i]).intersection(connection_atoms[r])):
            mini, maxi = dico_init_inter[(r,k)]
            center1, max1 = combination[i]
            center2, max2 = combination[j]
            test_res =  test_interval(center1, max1, center2, max2, mini, maxi)
            if test_res == False:
                tuple_ex = tuple((test_res, r, combination[i]))
                return tuple_ex
    return test_res

def test_interval(center1, max1,center2, max2, mini, maxi):
    #TODO
    ecart1 = np.linalg.norm(center1 - max1)
    ecart2 = np.linalg.norm(center2 - max2)
    norm = np.linalg.norm(center1 - center2)
    if (norm + ecart1 + ecart2 < mini or norm - ecart1 - ecart2 > maxi):
        return False
    return True

In [324]:
def test_connections(h_layer, sub=subf, connection_atoms=connection_atoms, multiproc=False):
    index = []
    for atom in sub.hierarchie[h_layer]:
        for atom2 in set(sub.hierarchie[h_layer]).intersection(connection_atoms[atom]):
            if atom2 < atom:
                continue
            index.append(atom)
            index.append(atom2)
    index = list(set(index))

    args, list_of_spaces = [], []
    for r in index:
        list_of_spaces.append(subf.position_atoms[r])

    all_combinations = list(product(*list_of_spaces))
    for combination in all_combinations:
        args.append((combination, index))

    if multiproc:
        nombre_processus = multiprocessing.cpu_count()
        with multiprocessing.Pool(processes=nombre_processus) as pool:
            results = pool.map(test_stack, args)
    else:
        results = []
        for arg in args:
            results.append(test_stack(arg))
    
    print(results)
    
    for res in results:
        if np.any(res[0] == 0):
            indices = np.where(res[0] == 0)
            for k in range(len(indices)):
                del subf.position_atoms[res[1]][indices[k] - k]


In [325]:
def initiate_intervalles_2(sub: sub_fc, connection_atoms, h_layer):
    args = []
    for i in sub.hierarchie[h_layer]:
        sub.position_atoms[i] = []
        hier = set(chain(*subf.hierarchie[:h_layer]))
        cst = list(hier.intersection(connection_atoms[i]))
        list_of_spaces = []
        for r in cst:
            list_of_spaces.append(subf.position_atoms[r])

        all_combinations = list(product(*list_of_spaces))
        for combination in all_combinations:
            args.append((cst, i, combination))
    
    nombre_processus = multiprocessing.cpu_count()
    with multiprocessing.Pool(processes=nombre_processus) as pool:
        result = pool.map(init_multiproc, args)
    for res in result:
        update_position_atoms_format2(res[0], res[1], res[2], subf = subf)


def init_multiproc(arg):
    cst, i, combination = arg
    li_ctc,li_f, li_constrains, mini, maxi, point = get_f_ctc_2(cst, i, combination)
    return (get_possibilities_from_inter(li_ctc,li_f, li_constrains, mini, maxi, point, i))


In [326]:
initiate_atoms(subf, dico_init_inter, connection_atoms)

[([2.882774399775327, 2.941018150233012] ; [3.043417883165111, 3.141592653589794] ; [0.3649765100670425, 0.7944196391783535]), ([2.882774399775327, 2.941018150233012] ; [2.94524311274043, 3.043417883165112] ; [0, 2.266067419099405]), ([2.882774399775327, 2.941018150233012] ; [2.84706834231575, 2.945243112740431] ; [0, 2.359191990759245]), ([2.882774399775327, 2.941018150233012] ; [2.748893571891068, 2.847068342315751] ; [0, 2.436809598816899]), ([2.882774399775327, 2.941018150233012] ; [2.650718801466387, 2.748893571891069] ; [0, 2.500106946441005]), ([2.882774399775327, 2.941018150233012] ; [2.552544031041707, 2.650718801466388] ; [0, 2.549648108511021]), ([2.882774399775327, 2.941018150233012] ; [2.454369260617025, 2.552544031041708] ; [0, 2.585970532356773]), ([2.882774399775327, 2.941018150233012] ; [2.356194490192344, 2.454369260617026] ; [0, 2.610007720040639]), ([2.882774399775327, 2.941018150233012] ; [2.258019719767663, 2.356194490192345] ; [0, 2.623311992971377]), ([2.8827743

In [327]:
for i in subf.position_atoms:
    print(i, len(subf.position_atoms[i]), end=' --- ')
print(subf.position_atoms[2][0])

def update_position_atoms_format(subf):
    for i in subf.position_atoms:
        if len(subf.position_atoms[i][0])<3:
            continue
        point = subf.origin_atom[i]
        for j in range(len(subf.position_atoms[i])):
            if len(subf.position_atoms[i][j])<3:
                continue
            r = sum(subf.position_atoms[i][j][0])/2
            theta = sum(subf.position_atoms[i][j][1])/2
            phi = sum(subf.position_atoms[i][j][2])/2
            subf.position_atoms[i][j] = (
                np.array([
                    r * math.sin(phi) * math.cos(theta),
                    r * math.sin(phi) * math.sin(theta),
                    r * math.cos(phi)
                ]) + point,
                np.array([
                    subf.position_atoms[i][j][0][1] * math.sin(subf.position_atoms[i][j][2][1]) * math.cos(subf.position_atoms[i][j][1][1]),
                    subf.position_atoms[i][j][0][1] * math.sin(subf.position_atoms[i][j][2][1]) * math.sin(subf.position_atoms[i][j][1][1]),
                    subf.position_atoms[i][j][0][1] * math.cos(subf.position_atoms[i][j][2][1])
                ]) + point,
            )

update_position_atoms_format(subf)

for i in subf.position_atoms:
    print(i, len(subf.position_atoms[i]), end=' --- ')
print(subf.position_atoms[2][0])

19 1 --- 18 1 --- 26 1 --- 2 32 --- 10 6 --- 44 42 --- 47 64 --- 49 64 --- 27 64 --- ([2.882774399775327, 2.941018150233012] ; [3.043417883165111, 3.141592653589794] ; [0.3649765100670425, 0.7944196391783535])
19 1 --- 18 1 --- 26 1 --- 2 32 --- 10 6 --- 44 42 --- 47 64 --- 49 64 --- 27 64 --- (array([-0.66933489, -3.4948851 ,  2.33638272]), array([-1.174493  , -3.57315066,  1.96097686]))


In [328]:
test_connections(h_layer=1, multiproc=False)


[True, True, (False, 47, (array([-3.8275964 ,  0.76384949,  0.58682482]), array([-4.09716678,  0.68264149,  0.21847481]))), (False, 47, (array([-3.76591825,  1.02233878,  0.56225811]), array([-4.09508384,  0.98317946,  0.05921156]))), True, True, True, True, True, True, True, True, (False, 47, (array([-2.09291704,  2.56724961,  0.70836186]), array([-2.31925219,  2.77890705,  0.31931026]))), (False, 47, (array([-1.84095037,  2.61455003,  0.74617823]), array([-2.02969066,  2.80014082,  0.45952634]))), (False, 47, (array([-1.59265388,  2.66609828,  0.74334597]), array([-1.75037899,  2.80885985,  0.55701106]))), (False, 47, (array([-1.33947933,  2.67651198,  0.76254842]), array([-1.47931203,  2.80940992,  0.61743072]))), (False, 47, (array([-1.08849698,  2.65037897,  0.7965328 ]), array([-1.21334624,  2.81841082,  0.62352058]))), (False, 47, (array([-0.84153718,  2.615547  ,  0.80987497]), array([-0.94564873,  2.82699236,  0.59061388]))), (False, 47, (array([-0.60824373,  2.52471856,  0.86

TypeError: 'bool' object is not subscriptable

In [ ]:
initiate_intervalles_2(subf, connection_atoms, 2)

In [ ]:
print(subf.hierarchie[2])
for i in subf.hierarchie[2]:
    print(i, len(subf.position_atoms[i]))

[39, 40, 41, 43, 45, 46, 16, 48]
39 1513
40 2065
41 76156
43 1858
45 188575
46 112783
16 2048
48 82974


In [ ]:
test_connections(h_layer=2, multiproc=True)

11


KeyboardInterrupt: 

In [ ]:
initiate_intervalles_2(subf, dico_init_inter, connection_atoms, 3) 

TypeError: initiate_intervalles_2() takes 3 positional arguments but 4 were given

In [ ]:
initiate_intervalles_2(subf, dico_init_inter, connection_atoms, 4)

In [ ]:
initiate_intervalles_2(subf, dico_init_inter, connection_atoms, 5)